In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.preprocessing import LabelEncoder

In [3]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from keras import Sequential
from keras.layers import Dense

In [6]:
from sklearn.pipeline import Pipeline

In [7]:
from tensorflow.keras.optimizers import Adam

In [8]:
import keras

In [9]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [10]:
train.head()

,model,year,motor_type,running,wheel,color,type,status,motor_volume,price
0,toyota,2022,petrol,3000 km,left,skyblue,sedan,excellent,2.0,24500
1,mercedes-benz,2014,petrol,132000 km,left,black,sedan,excellent,2.0,25500
2,kia,2018,petrol,95000 miles,left,other,sedan,excellent,2.0,11700
3,mercedes-benz,2002,petrol,137000 miles,left,golden,sedan,excellent,3.2,12000
4,mercedes-benz,2017,petrol,130000 km,left,black,sedan,good,2.0,26000


In [11]:
test.head()

,Id,model,year,motor_type,running,wheel,color,type,status,motor_volume
0,0,kia,2020,petrol,24000 km,left,black,sedan,excellent,2.0
1,1,nissan,2017,petrol,85000 miles,left,white,suv,excellent,2.0
2,2,hyundai,2021,petrol,30000 miles,left,white,sedan,excellent,2.0
3,3,kia,2018,petrol,53000 miles,left,silver,sedan,excellent,2.0
4,4,mercedes-benz,2003,petrol,230000 km,left,black,Universal,normal,1.8


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1642 entries, 0 to 1641
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   model         1642 non-null   object 
 1   year          1642 non-null   int64  
 2   motor_type    1642 non-null   object 
 3   running       1642 non-null   object 
 4   wheel         1642 non-null   object 
 5   color         1642 non-null   object 
 6   type          1642 non-null   object 
 7   status        1642 non-null   object 
 8   motor_volume  1642 non-null   float64
 9   price         1642 non-null   int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 128.4+ KB


In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411 entries, 0 to 410
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            411 non-null    int64  
 1   model         411 non-null    object 
 2   year          411 non-null    int64  
 3   motor_type    411 non-null    object 
 4   running       411 non-null    object 
 5   wheel         411 non-null    object 
 6   color         411 non-null    object 
 7   type          411 non-null    object 
 8   status        411 non-null    object 
 9   motor_volume  411 non-null    float64
dtypes: float64(1), int64(2), object(7)
memory usage: 32.2+ KB


In [14]:
test = test.drop('Id', axis = 1)

In [15]:
for i in ['model', 'motor_type', 'wheel', 'color', 'type', 'status']:
    print(i, set(test[i].unique()).difference(set(train[i].unique())))

model set()
motor_type set()
wheel {'right'}
color set()
type set()
status set()


In [16]:
train = train.drop('wheel', axis = 1)
test = test.drop('wheel', axis = 1)

In [17]:
label_encoders = {}

In [18]:
for i in ['model', 'motor_type', 'color', 'type', 'status']:
    label = LabelEncoder()
    train[i] = label.fit_transform(train[i])
    test[i] = label.transform(test[i])
    label_encoders[i] = label

In [19]:
train.running = train.running.apply(lambda p: float(p.split(' ')[0]) if p.split(' ')[-1] == 'km' else 1.6 * float(p.split(' ')[0]))
test.running = test.running.apply(lambda p: float(p.split(' ')[0]) if p.split(' ')[-1] == 'km' else 1.6 * float(p.split(' ')[0]))

In [22]:
train.head()

,model,year,motor_type,running,color,type,status,motor_volume,price
0,4,2022,3,3000.0,15,5,1,2.0,24500
1,2,2014,3,132000.0,1,5,1,2.0,25500
2,1,2018,3,152000.0,10,5,1,2.0,11700
3,2,2002,3,219200.0,6,5,1,3.2,12000
4,2,2017,3,130000.0,1,5,2,2.0,26000


In [23]:
test.head()

,model,year,motor_type,running,color,type,status,motor_volume
0,1,2020,3,24000.0,1,5,1,2.0
1,3,2017,3,136000.0,16,6,1,2.0
2,0,2021,3,48000.0,16,5,1,2.0
3,1,2018,3,84800.0,14,5,1,2.0
4,2,2003,3,230000.0,1,1,4,1.8


In [24]:
scaler = StandardScaler()

In [25]:
X = train.copy().drop('price', axis = 1)

In [26]:
Y = train.copy().price

In [27]:
Z = test.copy()

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [29]:
X_train.shape

(1313, 8)

In [30]:
X_test.shape

(329, 8)

In [31]:
model = Sequential()

model.add(Dense(units = 256, activation = 'relu', input_shape = (1, 8)))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 32, activation = 'relu'))
model.add(Dense(units = 16, activation = 'relu'))

model.add(Dense(units = 1, activation = 'linear'))

c:\users\user\desktop\python_smart_code\jupyter\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [33]:
X_train = X_train.reshape(-1, 1, 8)
X_test = X_test.reshape(-1, 1, 8)

In [34]:
X_train.shape

(1313, 1, 8)

In [35]:
X_test.shape

(329, 1, 8)

In [36]:
X = scaler.fit_transform(X)
Z = scaler.transform(Z)

In [37]:
X.shape

(1642, 8)

In [38]:
Z.shape

(411, 8)

In [39]:
X = X.reshape(-1, 1, 8)

In [40]:
Z = Z.reshape(-1, 1, 8)

In [41]:
model.compile(optimizer=Adam(learning_rate=0.005), loss='mae', metrics=['mae'])

In [42]:
model.fit(X, Y, epochs=400, batch_size=32, callbacks=[keras.callbacks.EarlyStopping(monitor='loss',patience=100)])

Epoch 1/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 14222.2969 - mae: 14222.2969
Epoch 2/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3826.3091 - mae: 3826.3091
Epoch 3/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3122.0168 - mae: 3122.0168
Epoch 4/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3096.0754 - mae: 3096.0754
Epoch 5/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3068.3562 - mae: 3068.3562
Epoch 6/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3054.3440 - mae: 3054.3440
Epoch 7/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2982.8066 - mae: 2982.8066
Epoch 8/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2940.0791 - mae: 2940.0791
Epoch 9/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3067.8550 - mae: 3067.8550
Epoch 10/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3005.0466 - mae: 3005.0466
Epoch 11/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 2952.9719 - mae: 2952.9719
Epoch 12/400
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/ste

In [49]:
def predict_price(input_list):

    columns = ['model', 'year', 'motor_type', 'running', 'wheel', 'color', 'type', 'status', 'motor_volume']
    df = pd.DataFrame([input_list], columns=columns)
    
    df = df.drop('wheel', axis=1)
    
    df['running'] = df['running'].apply(lambda p: float(p.split(' ')[0]) if p.split(' ')[-1] == 'km' else 1.6 * float(p.split(' ')[0]))

    for col in ['model', 'motor_type', 'color', 'type', 'status']:
        df[col] = label_encoders[col].transform(df[col])

    
    X_scaled = scaler.transform(df)
    
    X_scaled = X_scaled.reshape(1, 1, 8)

    return X_scaled

In [50]:
data = predict_price(['nissan', 2017, 'petrol',	'85000 miles', 'left', 'white', 'suv', 'excellent', 2.0])

In [51]:
data

array([[[ 0.79204149,  0.33319461,  0.26699337,  0.17698555,
          1.24997898,  1.08693988, -0.45353122, -0.1384163 ]]])

In [52]:
model.predict(data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


array([[[15763.271]]], dtype=float32)

In [53]:
import joblib

In [61]:
import pickle

In [55]:
model.save('model_files/model.h5')

In [63]:
with open('model_files/model2.pkl', 'wb') as file:
    pickle.dump(model, file)

In [59]:
for col, label in label_encoders.items():
    joblib.dump(label, f"model_files/label_{col}.pkl")

In [60]:
joblib.dump(scaler, "model_files/scaler.pkl")

['model_files/scaler.pkl']